In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
# @Date    : 2018-09-20 11:39:53
# @Author  : Guosheng Kang (guoshengkang@gmail.com)

import os
import  torch
import torchvision
from torchvision import datasets,transforms
from torch.autograd import Variable
import matplotlib.pylab as plt
%matplotlib inline

batch_size=64
transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize(mean=[0.5,0.5,0.5],std=[0.5,0.5,0.5])])

data_train=datasets.MNIST(root="./data/",transform=transform,train=True,download=True)
data_test=datasets.MNIST(root="./data/",transform=transform,train=False)

data_loader_train=torch.utils.data.DataLoader(dataset=data_train,batch_size=batch_size,shuffle=True)
data_loader_test=torch.utils.data.DataLoader(dataset=data_test,batch_size=batch_size,shuffle=True)
# print(len(data_loader_train),len(data_loader_test)) # 938 157

# 取部分数据来训练和测试
train_batch_num=10
test_batch_num=4
tmp_data_loader_train=[]
for _ in range(train_batch_num):
	tmp_data_loader_train.append(next(iter(data_loader_train)))
tmp_data_loader_test=[]
for _ in range(test_batch_num):
	tmp_data_loader_test.append(next(iter(data_loader_test)))
data_loader_train,data_loader_test=tmp_data_loader_train,tmp_data_loader_test

# 展示一张图片
# images,labels=next(iter(data_loader_train))
# img=torchvision.utils.make_grid(images)
# img=img.numpy().transpose(1,2,0)
# std=[0.5,0.5,0.5]
# mean=[0.5,0.5,0.5]
# img=img*std+mean
# print([labels[i] for i in range(64)])
# plt.imshow(img)
# plt.show()

class Model(torch.nn.Module):
	def __init__(self):
		super(Model,self).__init__()
		self.conv1=torch.nn.Sequential(
			torch.nn.Conv2d(1,64,kernel_size=3,stride=1,padding=1),
			torch.nn.ReLU(),
			torch.nn.Conv2d(64,128,kernel_size=3,stride=1,padding=1),
			torch.nn.ReLU(),
			torch.nn.MaxPool2d(stride=2,kernel_size=2)
			)
		self.dense=torch.nn.Sequential(
			torch.nn.Linear(14*14*128,1024),
			torch.nn.ReLU(),
			torch.nn.Dropout(p=0.5),
			torch.nn.Linear(1024,10)
			)
	def forward(self,x):
		x=self.conv1(x)
		x=x.view(-1,14*14*128)
		x=self.dense(x)
		return x

model=Model()
cost=torch.nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters())
# print(model)

n_epochs=5
for epoch in range(n_epochs):
	running_loss=0.0
	running_correct=0
	print("Epoch {}/{}".format(epoch,n_epochs))
	for index,data in enumerate(data_loader_train):
		X_train,y_train=data
		# print("size of X_train:{}".format(X_train.size()))
		X_train,y_train=Variable(X_train),Variable(y_train)
		outputs=model(X_train)
		_,pred=torch.max(outputs.data,1)
		optimizer.zero_grad()
		loss=cost(outputs,y_train)
		loss.backward()
		optimizer.step()
		running_loss+=loss.data[0]
		running_correct+=torch.sum(pred==y_train.data)
	testing_correct=0
	for data in data_loader_test:
		X_test,y_test=data
		X_test,y_test=Variable(X_test),Variable(y_test)
		outputs=model(X_test)
		_,pred=torch.max(outputs.data,1)
		testing_correct+=torch.sum(pred==y_test.data)
	print("Loss is:{:.4f},Train Accuracy is:{:.4f}%,Test Accuracy is:{:.4f}%".format(
		running_loss/(train_batch_num*batch_size),
		100*running_correct/(train_batch_num*batch_size),
		100*testing_correct/(test_batch_num*batch_size)))
	print("-"*30)


Epoch 0/5
Loss is:0.0414,Train Accuracy is:24.0625%,Test Accuracy is:60.5469%
------------------------------
Epoch 1/5
Loss is:0.0164,Train Accuracy is:72.6562%,Test Accuracy is:81.6406%
------------------------------
Epoch 2/5
Loss is:0.0075,Train Accuracy is:86.0938%,Test Accuracy is:82.0312%
------------------------------
Epoch 3/5
Loss is:0.0054,Train Accuracy is:87.9688%,Test Accuracy is:87.8906%
------------------------------
Epoch 4/5
Loss is:0.0031,Train Accuracy is:94.3750%,Test Accuracy is:85.9375%
------------------------------
